In [ ]:
import cobra
import pandas as pd  # Add this import statement

# Load your COBRA model
model = cobra.io.read_sbml_model("Megasphaera_sp_MJR8396C.xml")

# Define the list of essential amino acids
essential_aa = ["EX_cpd00119_e0", "EX_cpd00060_e0", "EX_cpd00084_e0"]

# Define the list of exchange reactions for amino acids in the model
amino_acid_exchanges = [
    'EX_cpd00119_e0', 'EX_cpd00060_e0', 'EX_cpd00084_e0', 'EX_cpd00023_e0',
    'EX_cpd00041_e0', 'EX_cpd00054_e0', 'EX_cpd00051_e0', 'EX_cpd00161_e0',
    'EX_cpd00035_e0', 'EX_cpd00033_e0', 'EX_cpd00066_e0', 'EX_cpd00069_e0',
    'EX_cpd00107_e0', 'EX_cpd00156_e0', 'EX_cpd00065_e0', 'EX_cpd00039_e0',
    'EX_cpd00322_e0', 'EX_cpd00129_e0', 'EX_cpd00132_e0', 'EX_cpd00064_e0', 'EX_cpd00027_e0', 'EX_cpd00053_e0'
]

# Dictionary to store growth rates for each pair
pair_growth_rates = {}

# Iterate through all pairs of amino acids
for i in range(len(amino_acid_exchanges)):
    for j in range(i+1, len(amino_acid_exchanges)):
        # Reset model
        model_copy = model.copy()
        
        # Set objective to maximize biomass production
        model_copy.objective = model_copy.reactions.get_by_id('EX_cpd11416_c0')
        
        # Remove all carbohydrates from the medium
        for reaction_id in [
            'EX_cpd00154_e0', 'EX_cpd00138_e0', 'EX_cpd03198_e0', 'EX_cpd00224_e0',
            'EX_cpd00185_e0', 'EX_cpd00108_e0', 'EX_cpd00609_e0', 'EX_cpd00139_e0',
            'EX_cpd00280_e0', 'EX_cpd00653_e0', 'EX_cpd00314_e0', 'EX_cpd00588_e0',
            'EX_cpd00306_e0', 'EX_cpd00179_e0', 'EX_cpd00076_e0', 'EX_cpd00082_e0',
            'EX_cpd00158_e0'
        ]:
            model_copy.reactions.get_by_id(reaction_id).lower_bound = 0
        
        # Set acetate in the medium
        model_copy.reactions.get_by_id('EX_cpd00029_e0').lower_bound =  0
        
        # Get exchange reactions for the pair of amino acids
        exchange_i = amino_acid_exchanges[i]
        exchange_j = amino_acid_exchanges[j]
        
        # Set lower bound of exchange reactions to -10 mmol/gDW/hr if in a pair, otherwise to 0 or -0.1
        for aa in amino_acid_exchanges:
            if aa not in [exchange_i, exchange_j]:
                if aa in essential_aa:
                    model_copy.reactions.get_by_id(aa).lower_bound = -0.1
                else:
                    model_copy.reactions.get_by_id(aa).lower_bound = 0
            else:
                model_copy.reactions.get_by_id(aa).lower_bound = -10
        
        # Optimize model with modified boundaries
        pFBA_solution = model_copy.optimize()
        
        # Store growth rate for the pair
        pair_growth_rates[(exchange_i, exchange_j)] = pFBA_solution.objective_value

# Create DataFrame from dictionary
pair_growth_rates_df = pd.DataFrame(columns=amino_acid_exchanges)
for exchange_i in amino_acid_exchanges:
    row = []
    for exchange_j in amino_acid_exchanges:
        row.append(pair_growth_rates.get((exchange_i, exchange_j), None))
    pair_growth_rates_df.loc[exchange_i] = row

# Write DataFrame to Excel file
pair_growth_rates_df.to_excel("AApairs2.xlsx")
